In [1]:
# Imports 
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import json                                                               
import numpy as np

import shutil
import sys
import os.path

#install pyomo module if it's not already present in the libraries
if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

## Use the cbc solver (right now it is not working will look for another solver )
'''if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc")'''



import pyomo.environ as py

'if not (shutil.which("cbc") or os.path.isfile("cbc")):\n    if "google.colab" in sys.modules:\n        !apt-get install -y -qq coinor-cbc\n    else:\n        try:\n            !conda install -c conda-forge coincbc \n        except:\n            pass\n\nassert(shutil.which("cbc") or os.path.isfile("cbc")'

In [3]:
demand_data_path = os.path.join(r'C:\Users\Agnes\OneDrive\Documents\Internship\Python scripts', 'demand_data.json')
price_data_path = os.path.join(r'C:\Users\Agnes\OneDrive\Documents\Internship\Python scripts', 'price_data.txt')
qty_data_path = os.path.join(r'C:\Users\Agnes\OneDrive\Documents\Internship\Python scripts', 'qty_data.txt')
farms_path = os.path.join(r'C:\Users\Agnes\OneDrive\Documents\Internship\Python scripts', 'farms.json')
produce_path =  os.path.join(r'C:\Users\Agnes\OneDrive\Documents\Internship\Python scripts', 'produce.json')
organic_data_path = os.path.join(r'C:\Users\Agnes\OneDrive\Documents\Internship\Python scripts', 'organic_data.txt')
local_data_path = os.path.join(r'C:\Users\Agnes\OneDrive\Documents\Internship\Python scripts', 'local_data.txt')


with open(farms_path,"r") as f:
    farms = json.load(f)
    
with open(produce_path,"r") as f:
    produce = json.load(f)
    
with open(demand_data_path,"r") as f:
    demand_data = json.load(f)
    
dic = ''
with open(price_data_path,'r') as f:
         for i in f.readlines():
            dic=i #string
price_data = eval(dic) # this is orignal dict with instace dict    

dic = ''
with open(qty_data_path,'r') as f:
         for i in f.readlines():
            dic=i #string
qty_data = eval(dic) # this is orignal dict with instace dict

dic = ''
with open(organic_data_path,'r') as f:
         for i in f.readlines():
            dic=i #string
organic_data = eval(dic) # this is orignal dict with instace dict

dic = ''
with open(local_data_path,'r') as f:
         for i in f.readlines():
            dic=i #string
local_data = eval(dic) # this is orignal dict with instace dict

## Creating our Model 
We would be using the Abstract model. We would define all the different blocks of our function. Then afterwards we would import the data.

model = py.ConcreteModel()

## Define the different sets 
model.PRODUCE = py.Set(initialize = produce)     # set of all produce 
model.FARMS =   py.Set(initialize = farms)   # set of all the farms  
model.local =   py.Set(initialize = local_data)
model.organic = py.Set(model.PRODUCE,model.FARMS,initialize = organic_data)    # set of all the organic farms '''

## Define Parameters
#model.c = py.Param(within = NonNegativeReals,data = initialize())
model.p = py.Param(model.PRODUCE,model.FARMS,initialize = price_data)
model.a = py.Param(model.PRODUCE,model.FARMS, initialize = qty_data)
model.b = py.Param(model.PRODUCE,initialize = demand_data)
model.alpha = py.Param(initialize = .85)
model.beta = py.Param(initialize = 1.0)


221## Declare decision variables 
model.x = py.Var( model.PRODUCE*model.FARMS, domain = py.NonNegativeReals,initialize = 0)#qty of produce of type i bought from farm j 
model.s = py.Var(model.PRODUCE, model.FARMS, domain = py.Binary,initialize = 1)

## Define constraints

# Demand constraints 
def demand_constraint(model,i):
    return sum(model.x[i,j] for j in model.FARMS) == model.b[i] 
model.demand = py.Constraint (model.PRODUCE, rule = demand_constraint)

# Availability constraints 
def availability_constraint(model,i,j):
    return (model.x[i,j] - model.a[i,j]*model.s[i,j] <= 0 )
model.available = py.Constraint(model.PRODUCE, model.FARMS, rule = availability_constraint)
#model.available = py.Constraint(model.PRODUCE, model.FARMS, rule = lambda model,i,j : model.x[i,j] <=  model.a[i,j])


## Organic constraints:
def org_constraint(model):
    return (sum(model.x[i,j] for (i,j) in model.organic) >= sum(model.x[i,j] for i in model.PRODUCE for j in model.FARMS)*model.beta)
model.org = py.Constraint(rule = org_constraint)

## Local constraints
def local_constraint(model):
     return (sum(model.x[i,j] for (i,j) in model.local) >= sum(model.x[i,j] for i in model.PRODUCE for j in model.FARMS)*model.alpha)
model.loc = py.Constraint(rule = local_constraint)

## Declare objective 
def obj_expression(model):
    return ( sum(model.p[i, j] * model.x[i, j] for i in model.PRODUCE for j in model.FARMS))
model.profit = py.Objective(rule = obj_expression)

# unique constraints 
def unique_constraint(model, i):
    return (sum(model.s[i,j] for j in model.FARMS )== 1)
model.unique = py.Constraint(model.PRODUCE, rule = unique_constraint)

## Local constraints 
def loc_constraint(model):
    return (sum(model.x[i,j] for i in model.organic) <= model.x[i,j]*model.alpha)

solver = py.SolverFactory('glpk')
result = solver.solve(model)

result.write()

In [15]:
model.display()

Model unknown

  Variables:
    x : Size=36, Index=x_index
        Key                                        : Lower : Value : Upper : Fixed : Stale : Domain
            ('Allium, Green Onion', 'FarmToTable') :     0 :  55.0 :  None : False : False : NonNegativeReals
              ('Allium, Green Onion', 'Farmshare') :     0 :   0.0 :  None : False : False : NonNegativeReals
                    ('Allium, Green Onion', 'JBG') :     0 :   0.0 :  None : False : False : NonNegativeReals
                ('Allium, Green Onion', 'Segovia') :     0 :   0.0 :  None : False : False : NonNegativeReals
                          ('Beets', 'FarmToTable') :     0 :  50.0 :  None : False : False : NonNegativeReals
                            ('Beets', 'Farmshare') :     0 :  55.0 :  None : False : False : NonNegativeReals
                                  ('Beets', 'JBG') :     0 :   0.0 :  None : False : False : NonNegativeReals
                              ('Beets', 'Segovia') :     0 :   0.0 :  N

Model unknown

  Variables:
    x : Size=36, Index=x_index
        Key                                        : Lower : Value : Upper : Fixed : Stale : Domain
            ('Allium, Green Onion', 'FarmToTable') :     0 :  55.0 :  None : False : False : NonNegativeReals
              ('Allium, Green Onion', 'Farmshare') :     0 :   0.0 :  None : False : False : NonNegativeReals
                    ('Allium, Green Onion', 'JBG') :     0 :   0.0 :  None : False : False : NonNegativeReals
                ('Allium, Green Onion', 'Segovia') :     0 :   0.0 :  None : False : False : NonNegativeReals
                          ('Beets', 'FarmToTable') :     0 :  50.0 :  None : False : False : NonNegativeReals
                            ('Beets', 'Farmshare') :     0 :  55.0 :  None : False : False : NonNegativeReals
                                  ('Beets', 'JBG') :     0 :   0.0 :  None : False : False : NonNegativeReals
                              ('Beets', 'Segovia') :     0 :   0.0 :  N

In [16]:
model.local.pprint()

local : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     2 :    Any :   27 : {('Allium, Green Onion', 'JBG'), ('Allium, Green Onion', 'FarmToTable'), ('Allium, Green Onion', 'Farmshare'), ('Beets', 'JBG'), ('Beets', 'FarmToTable'), ('Beets', 'Farmshare'), ('Carrots', 'JBG'), ('Carrots', 'FarmToTable'), ('Carrots', 'Farmshare'), ('Collard Greens', 'JBG'), ('Collard Greens', 'FarmToTable'), ('Collard Greens', 'Farmshare'), ('Kale, Lacinato', 'JBG'), ('Kale, Lacinato', 'FarmToTable'), ('Kale, Lacinato', 'Farmshare'), ('Radish, Average Variety', 'JBG'), ('Radish, Average Variety', 'FarmToTable'), ('Radish, Average Variety', 'Farmshare'), ('Spinach', 'JBG'), ('Spinach', 'FarmToTable'), ('Spinach', 'Farmshare'), ('Squash, Summer', 'JBG'), ('Squash, Summer', 'FarmToTable'), ('Squash, Summer', 'Farmshare'), ('Swiss Chard', 'JBG'), ('Swiss Chard', 'FarmToTable'), ('Swiss Chard', 'Farmshare')}
local : Size=1, Index=None, Ordered=Insertion
    Key  :

In [17]:
dic = {}
for i in model.PRODUCE:
    for j in model.FARMS:
            dic[(i,j)] = py.value(model.x[i,j])
with open('x.txt','w')as f:
    for i in model.PRODUCE:
        for j in model.FARMS:
            f.write(str(dic))

In [18]:
with open('x.csv','w')as f:
    f.write('Produce,Farms,Value,\n')
    for i in model.PRODUCE:
        for j in model.FARMS:
            f.write( '%s,%s,%s,\n' % (i,j,str(model.x[i,j].value)))